In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [4]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)
# ---
class VGG16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        vgg = models.vgg16(pretrained=pretrained)
        self.encoder = vgg.features
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_transform = transforms.Compose([
    transforms.Resize((512, 1024)),             
    # transforms.RandomHorizontalFlip(p=0.5),      
    transforms.ColorJitter(brightness=0.2),      
    transforms.ToTensor(),
])
    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        train_transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = VGG16Seg(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(5):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("jitter_submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [05:17<00:00,  1.43it/s, loss=0.3128]


[Epoch 1] 평균 손실: 0.3114


Epoch 2: 100%|██████████| 454/454 [05:18<00:00,  1.43it/s, loss=0.2221]


[Epoch 2] 평균 손실: 0.2571


...

[Epoch 4] 평균 손실: 0.2391


Epoch 5: 100%|██████████| 454/454 [05:17<00:00,  1.43it/s, loss=0.2268]


[Epoch 5] 평균 손실: 0.2347
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [02:32<00:00, 18.20it/s]


✅ submission.csv 파일이 저장되었습니다!


In [16]:
torch.cuda.empty_cache()

In [2]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)
# ---
class VGG16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        vgg = models.vgg16(pretrained=pretrained)
        self.encoder = vgg.features
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_transform = transforms.Compose([
    transforms.Resize((512, 1024)),             
    transforms.RandomHorizontalFlip(p=0.5),      
    # transforms.ColorJitter(brightness=0.2),      
    transforms.ToTensor(),
])
    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        train_transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = VGG16Seg(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(5):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [05:15<00:00,  1.44it/s, loss=0.3691]


[Epoch 1] 평균 손실: 0.4292


Epoch 2: 100%|██████████| 454/454 [05:16<00:00,  1.43it/s, loss=0.3573]


[Epoch 2] 평균 손실: 0.3867


...

[Epoch 4] 평균 손실: 0.3586


Epoch 5: 100%|██████████| 454/454 [05:16<00:00,  1.43it/s, loss=0.3923]


[Epoch 5] 평균 손실: 0.3497
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [02:31<00:00, 18.31it/s]


✅ submission.csv 파일이 저장되었습니다!


In [3]:
# 필요한 라이브러리 임포트
import os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models

# ---------- Dice Loss 함수 정의 ----------
# 예측과 실제 정답 간의 겹치는 정도를 측정하는 Dice Loss 함수
def dice_loss(pred, target, smooth=1e-6):
    pred = torch.softmax(pred, dim=1)[:, 1, :, :]  # 클래스 1에 대한 softmax 확률
    target = (target == 1).float()                 # target도 1인 픽셀만 선택
    intersection = (pred * target).sum(dim=(1, 2)) # 교집합
    union = pred.sum(dim=(1, 2)) + target.sum(dim=(1, 2)) # 합집합
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice.mean()

# ---------- 데이터셋 클래스 ----------
class LaneDataset(Dataset):
    def __init__(self, img_dir, mask_dir=None, transform=None):
        self.img_dir = img_dir      # 이미지 디렉토리
        self.mask_dir = mask_dir    # 마스크 디렉토리
        self.images = sorted(os.listdir(img_dir))  # 이미지 파일 정렬
        self.transform = transform

    def __len__(self): 
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        pil_img = Image.open(img_path).convert('RGB')
        original_size = pil_img.size  # (너비, 높이)
        image = self.transform(pil_img)

        if self.mask_dir:
            # 학습/검증 시 마스크도 함께 불러옴
            mask_path = os.path.join(self.mask_dir, img_name)
            mask = Image.open(mask_path).convert('L').resize((1024, 512))
            mask = (np.array(mask) > 127).astype(np.uint8)  # 이진화
            mask = torch.tensor(mask, dtype=torch.long)
            return image, mask
        else:
            # 테스트 시 마스크 없음 → 파일명과 원본 크기 반환
            return image, img_name, original_size

# ---------- 간단한 CNN 모델 정의 ----------
class SimpleCNNModel(nn.Module):
    def __init__(self, n_class):
        super().__init__()
        # 인코더 (특징 추출)
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128, 128, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, 3, padding=1), nn.ReLU(),
            nn.Conv2d(256, 256, 3, padding=1), nn.ReLU()
        )
        # 디코더 (업샘플링)
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 128, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(128, 128, 2, 2), nn.ReLU(),
            nn.Conv2d(128, 64, 3, padding=1), nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 2, 2), nn.ReLU(),
            nn.Conv2d(64, 32, 3, padding=1), nn.ReLU()
        )
        # 클래스 분류기
        self.classifier = nn.Conv2d(32, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return self.classifier(x)
# ---
class VGG16Seg(nn.Module):
    def __init__(self, n_class=2, pretrained=True):
        super().__init__()
        vgg = models.vgg16(pretrained=pretrained)
        self.encoder = vgg.features
        self.classifier = nn.Conv2d(512, n_class, kernel_size=1)

    def forward(self, x):
        x = self.encoder(x)
        x = self.classifier(x)  # [B, n_class, H/32, W/32]
        # Upsample back to input size
        return nn.functional.interpolate(x, scale_factor=32, mode='bilinear', align_corners=False)
# ---------- 마스크를 RLE로 인코딩하는 함수 ----------
def mask_to_rle(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# ---------- 메인 함수 ----------
def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    train_transform = transforms.Compose([
    transforms.Resize((512, 1024)),                 
    transforms.ToTensor(),
])
    # 이미지 전처리 정의
    transform = transforms.Compose([
        transforms.Resize((512, 1024)),
        transforms.ToTensor()
    ])

    # --- 학습 시작 ---
    train_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/train/train/frames",
        "/kaggle/input/lanesegmentationchallenge/train/train/lane-masks",
        train_transform
    )
    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4)

    model = VGG16Seg(n_class=2).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
    ce_loss_fn = nn.CrossEntropyLoss()

    print("▶️ 모델 학습 시작...")
    for epoch in range(5):
        model.train()
        total_loss = 0
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}")
        for imgs, masks in pbar:
            imgs, masks = imgs.to(device), masks.to(device)
            outputs = model(imgs)
            ce = ce_loss_fn(outputs, masks)
            d  = dice_loss(outputs, masks)
            loss = 0.5 * ce + 0.5 * d  
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            pbar.set_postfix(loss=f"{loss.item():.4f}")
        print(f"[Epoch {epoch+1}] 평균 손실: {total_loss / len(train_loader):.4f}")

    torch.save(model.state_dict(), "simple_cnn_model.pth")
    print("💾 모델이 simple_cnn_model.pth로 저장되었습니다.")

    # --- 테스트 이미지에 대한 추론 및 제출 파일 생성 ---
    print("🧪 submission.csv 생성 시작...")
    test_dataset = LaneDataset(
        "/kaggle/input/lanesegmentationchallenge/test/test/frames",
        None, transform
    )
    test_loader = DataLoader(test_dataset, batch_size=1)
    model.eval()

    results = []
    with torch.no_grad():
        for imgs, fnames, orig_size in tqdm(test_loader, desc="추론 중"):
            imgs = imgs.to(device)
            output = model(imgs)
            pred_mask = output.argmax(1).squeeze(0).cpu().numpy().astype(np.uint8)
            pred_pil = Image.fromarray(pred_mask)
            W, H = orig_size
            resized_mask = pred_pil.resize((W, H), resample=Image.NEAREST)
            binary = np.array(resized_mask) > 0
            rle = mask_to_rle(binary)
            results.append({'filename': fnames[0], 'rle': rle})

    pd.DataFrame(results).to_csv("origin_submission.csv", index=False)
    print("✅ submission.csv 파일이 저장되었습니다!")

# 프로그램 실행 진입점
if __name__ == "__main__":
    main()


▶️ 모델 학습 시작...


Epoch 1: 100%|██████████| 454/454 [05:16<00:00,  1.43it/s, loss=0.3021]


[Epoch 1] 평균 손실: 0.3033


Epoch 2: 100%|██████████| 454/454 [05:17<00:00,  1.43it/s, loss=0.2403]


[Epoch 2] 평균 손실: 0.2557


...

[Epoch 4] 평균 손실: 0.2386


Epoch 5: 100%|██████████| 454/454 [05:17<00:00,  1.43it/s, loss=0.2365]


[Epoch 5] 평균 손실: 0.2335
💾 모델이 simple_cnn_model.pth로 저장되었습니다.
🧪 submission.csv 생성 시작...


추론 중: 100%|██████████| 2782/2782 [02:32<00:00, 18.27it/s]


✅ submission.csv 파일이 저장되었습니다!
